# Projeto: Ler, coletar e exportar informações importantes de arquivo xml - Nota Fiscal.

### Objetivos:
* Ler arquivo de nota fiscal em xml utilizando python;
* Padrões das notas fiscais utilizadas: DANFE, Nota carioca(padrão município);
* Extrair informações importantes como tipo da nota fiscal, se é produto ou serviço, valor, cpnj/cpf do comprador, produtos, serviços prestados;
* Criar funções para facilitar a leitura e identificação dos outros arquivos xml de nota fiscal que estão na mesma pasta;
* Colocar essas informações em um dataframe e exportar para excel;

#### Informações importantes:
* A IDE utilizada foi o **Jupyter Notebook**;
* Os arquivos se encontram na mesma pasta onde o código foi realizado, caso os arquivos estejam em outra pasta, é necessário adaptar o código para passar o caminho completo dos arquivos;

### Importar as bibliotecas que serão utilizadas:

In [166]:
# Caso não possua a biblioteca abaixo, instale.
#!pip install xmltodict 

In [167]:
import xmltodict #Biblioteca que transforma arquivo xml para dicionário;
import pandas as pd
import os 
import pprint

In [168]:
#Abrir a nota fiscal e ler
with open('NFs Finais/DANFEBrota.xml', 'rb') as arquivo:
    documento = xmltodict.parse(arquivo)

pprint.pprint(documento) #Consegue visualizar as informações contidas em um dicionário em forma de estrutura de árvore

{'nfeProc': {'@versao': '4.00',
             '@xmlns': 'http://www.portalfiscal.inf.br/nfe',
             'NFe': {'@xmlns': 'http://www.portalfiscal.inf.br/nfe',
                     'Signature': {'@xmlns': 'http://www.w3.org/2000/09/xmldsig#',
                                   'KeyInfo': {'X509Data': {'X509Certificate': 'MIIH7zCCBdegAwIBAgIIdwS2COgNxIEwDQYJKoaaaELBQAwczELMaaGA1UEBhMCQlIxEzARBgNVBAoTCklDUC1CcmFzaWwxNjA0BgNVBAsTLVNlY3JldGFyaWEgZGEgUmVjZWl0YSBGZWRlcmFsIGRvIEJyYXNpbCAtIFJGQjEXMBUGA1UEAxMOQUMgTElOSyBSRkIgdjIwHhcNMjEwODI0MTE1ODQ1WhcNMjIwODI0MTE1ODQ1WjCCAQwxCzAJBgNVBAYTAkJSMRMwEQYDVQQKEwpJQ1AtQnJhc2lsMQswCQYDVQQIEwJSSjEXMBUGA1UEBxMOUklPIERFIEpBTkVJUk8xFzAVBgNVBAsTDjEyNTE3NzA0MDAwMTE1MTYwNAYDVQQLEy1TZWNyZXRhcmlhIGRhIFJlY2VpdGEgRmVkZXJhbCBkbyBCcmFzaWwgLSBSRkIxFjAUBgNVBAsTDVJGQiBlLUNOUEogQTExGTAXBgNVBAsTEHZpZGVvY29uZmVyZW5jaWExPjA8BgNVBAMTNUJST1RBIENPTVBBTlkgQ09NRVJDSU8gREUgUExBTlRBUyBMVERBOjM2ODgyMTk1MDAwMTk4MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAsVQ9+aXdG6t4nl/5M0nkbI/j

In [169]:
# Coletar informações de notas fiscais no padrão DANFE e guardar em uma função:
def ler_xml_danfe(nota_fiscal):
    
    with open(f'{nota_fiscal}', 'rb') as arquivo:
        documento = xmltodict.parse(arquivo)

    dict_nota_fiscal = documento['nfeProc']['NFe']['infNFe']

    def coletar_cpf_cnpj(nota, pessoa = 'dest'):
        try:
            return nota[pessoa]['CPF']
        except:
            return nota[pessoa]['CNPJ']


    valor_total = dict_nota_fiscal['total']['ICMSTot']['vProd']
    valor_nota_fiscal = dict_nota_fiscal['total']['ICMSTot']['vNF']
    total_desconto = dict_nota_fiscal['total']['ICMSTot']['vDesc']
    cnpj_cpf_vendeu = coletar_cpf_cnpj(dict_nota_fiscal, pessoa = 'emit')
    nome_vendeu = dict_nota_fiscal['emit']['xNome']
    cpf_cnpj_comprou = coletar_cpf_cnpj(dict_nota_fiscal)
    nome_comprou = dict_nota_fiscal['dest']['xNome']
    produtos = dict_nota_fiscal['det']
    lista_produtos = []

    for produto in produtos:
        nome_produto = produto['prod']['xProd']
        valor_produto = produto['prod']['vProd']
        lista_produtos.append((nome_produto, valor_produto))

    resposta = {'Tipo': 'Produto',
               'Total R$': float(valor_total),
               'Desconto Total R$': float(total_desconto),
               'Total da Nota Fiscal R$': float(valor_nota_fiscal),
               'CPF/CNPJ vendedor': cnpj_cpf_vendeu,
               'Nome vendedor': nome_vendeu,
               'CPF/CNPJ comprador': cpf_cnpj_comprou,
               'Nome comprador': nome_comprou,
               'Lista de Produtos/Serviços': lista_produtos}

    return resposta

In [170]:
# Coletar informações de notas fiscais no padrão Nota carioca e guardar em uma função:
def ler_notafiscal_servico(nota_fiscal):
    
    with open(f'{nota_fiscal}', 'rb') as arquivo:
        documento = xmltodict.parse(arquivo)

    dict_nota_fiscal = documento['ConsultarNfseResposta']['ListaNfse']['CompNfse']['Nfse']['InfNfse']

    def coletar_cpf_cnpj(nota, pessoa = 'Tomador'):
        try:
            return nota[pessoa+'Servico']['Identificacao'+pessoa]['CpfCnpj']['Cnpj']
        except:
            return nota[pessoa+'Servico']['Identificacao'+pessoa]['CpfCnpj']['Cpf']


    valor_total = dict_nota_fiscal['Servico']['Valores']['ValorServicos']
    valor_nota_fiscal = dict_nota_fiscal['Servico']['Valores']['ValorLiquidoNfse']
    total_desconto = float(valor_total) - float(valor_nota_fiscal)
    cnpj_vendeu = dict_nota_fiscal['PrestadorServico']['IdentificacaoPrestador']['Cnpj']
    nome_vendeu = dict_nota_fiscal['PrestadorServico']['RazaoSocial']
    cpf_cnpj_comprou = coletar_cpf_cnpj(dict_nota_fiscal)
    nome_comprou = dict_nota_fiscal['TomadorServico']['RazaoSocial']
    servico = dict_nota_fiscal['Servico']['Discriminacao']

    resposta = {'Tipo': 'Serviço',
               'Total R$': float(valor_total),
               'Desconto Total R$': float(total_desconto),
               'Total da Nota Fiscal R$': float(valor_nota_fiscal),
               'CPF/CNPJ vendedor': cnpj_cpf_vendeu,
               'Nome vendedor': nome_vendeu,
               'CPF/CNPJ comprador': cpf_cnpj_comprou,
               'Nome comprador': nome_comprou,
               'Lista de Produtos/Serviços': [servico]}

    return resposta

In [171]:
# Listar os arquivos que estão na pasta e usar para percorrer toda a pasta em busca de notas fiscais no padrão 
#definido

lista_arquivos = os.listdir('NFs Finais')

lista_notas_fiscais = []
for arquivo in lista_arquivos:
    if '.xml' in arquivo:
        if 'DANFE' in arquivo:
            lista_notas_fiscais.append(ler_xml_danfe(f'NFs Finais/{arquivo}'))
    
        else: 
            lista_notas_fiscais.append(ler_notafiscal_servico(f'NFs Finais/{arquivo}'))
            

In [172]:
#Colocar em um dataframe as informações das notas fiscais para criar uma base de dados de notas fiscais:
df_notas_fiscais = pd.DataFrame.from_records(lista_notas_fiscais)
df_notas_fiscais

,Tipo,Total R$,Desconto Total R$,Total da Nota Fiscal R$,CPF/CNPJ vendedor,Nome vendedor,CPF/CNPJ comprador,Nome comprador,Lista de Produtos/Serviços
0,Produto,449.00,75.0,374.17,36882195000279,BROTA COMPANY COMERCIO DE PLANTAS LTDA,10000000000,Lira da Hashtag,"[(b.box led, 389.00), (Tomilho Serpilho, 10.00..."
1,Produto,769.17,0.0,849.00,60409075055054,NESTLE BRASIL LTDA,11122233344,Lira da Hashtag,"[(ESSENZA Mini C30 Metal 110V, 534.82), (NOMAD..."
2,Serviço,4500.00,0.0,4500.00,36882195000279,LIRA BOLADO NO XML,26344392000108,HASHTAG TREINAMENTOS LTDA,[Nota referente aos serviços realizados no mês...


* OBS: Quando acontece dos valores da coluna: "Total RS" serem menores que o da coluna: "Total da Nota Fiscal RS", significa que houve impostos sobre o total, acarretando no final ao total da nota fiscal um valor mais alto;

In [173]:
df_notas_fiscais.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Tipo                        3 non-null      object 
 1   Total R$                    3 non-null      float64
 2   Desconto Total R$           3 non-null      float64
 3   Total da Nota Fiscal R$     3 non-null      float64
 4   CPF/CNPJ vendedor           3 non-null      object 
 5   Nome vendedor               3 non-null      object 
 6   CPF/CNPJ comprador          3 non-null      object 
 7   Nome comprador              3 non-null      object 
 8   Lista de Produtos/Serviços  3 non-null      object 
dtypes: float64(3), object(6)
memory usage: 344.0+ bytes
